<a href="https://colab.research.google.com/github/mengyaoo/FNL_GenesSelection/blob/main/01_TC1_data_process.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from __future__ import print_function

import os, sys, gzip, glob, json, time, argparse
import pandas as pd
import numpy as np
from pandas.io.json import json_normalize

from pandas.io.json import json_normalize
from keras.utils import to_categorical
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler, MinMaxScaler, MaxAbsScaler
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

from keras.utils import to_categorical
from keras import backend as K
from keras.layers import Input, Dense, Dropout, Activation, Conv1D, MaxPooling1D, Flatten
from keras import optimizers
from keras.optimizers import SGD, Adam, RMSprop
from keras.models import Sequential, Model, model_from_json, model_from_yaml
from keras.utils import np_utils
from keras.callbacks import ModelCheckpoint, CSVLogger, ReduceLROnPlateau



# Data Preparation   

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# read the merged FPKM file 
df_FPKM_UQ = pd.read_csv("/content/drive/My Drive/FNL_TC1/Merged_FPKM-UQ.tsv", low_memory=False, sep="\t")

In [ ]:
df_FPKM_UQ 

,Unnamed: 0,gene_name,TCGA-04-1331-01A-01R-1569-13,TCGA-04-1332-01A-01R-1564-13,TCGA-04-1338-01A-01R-1564-13,TCGA-04-1341-01A-01R-1564-13,TCGA-04-1343-01A-01R-1564-13,TCGA-04-1347-01A-01R-1564-13,TCGA-04-1356-01A-01R-1569-13,TCGA-04-1357-01A-01R-1565-13,TCGA-04-1362-01A-01R-1565-13,TCGA-04-1364-01A-01R-1565-13,TCGA-04-1365-01A-01R-1565-13,TCGA-04-1514-01A-01R-1566-13,TCGA-04-1519-01A-01R-1565-13,TCGA-04-1530-01A-02R-1569-13,TCGA-04-1536-01A-01R-1566-13,TCGA-04-1651-01A-01R-1567-13,TCGA-04-1655-01A-01R-1566-13,TCGA-05-4249-01A-01R-1107-07,TCGA-05-4382-01A-01R-1206-07,TCGA-05-4384-01A-01R-1755-07,TCGA-05-4389-01A-01R-1206-07,TCGA-05-4390-01A-02R-1755-07,TCGA-05-4396-01A-21R-1858-07,TCGA-05-4397-01A-01R-1206-07,TCGA-05-4398-01A-01R-1206-07,TCGA-05-4403-01A-01R-1206-07,TCGA-05-4410-01A-21R-1858-07,TCGA-05-4415-01A-22R-1858-07,TCGA-05-4417-01A-22R-1858-07,TCGA-05-4418-01A-01R-1206-07,TCGA-05-4422-01A-01R-1206-07,TCGA-05-4430-01A-02R-1206-07,TCGA-05-4432-01A-01R-1206-07,TCGA-05-4433-01A-22R-1858-07,TCGA-05-4434-01A-01R-1206-07,TCGA-05-5420-01A-01R-1628-07,TCGA-05-5423-01A-01R-1628-07,TCGA-05-5428-01A-01R-1628-07,...,TCGA-ZG-A9L2-01A-31R-A41O-07,TCGA-ZG-A9L6-01A-11R-A41O-07,TCGA-ZG-A9L9-01A-11R-A41O-07,TCGA-ZG-A9LB-01A-11R-A41O-07,TCGA-ZG-A9LM-01A-11R-A41O-07,TCGA-ZG-A9LN-01A-11R-A41O-07,TCGA-ZG-A9LS-01A-12R-A41O-07,TCGA-ZG-A9LZ-01A-11R-A41O-07,TCGA-ZG-A9M4-01A-11R-A41O-07,TCGA-ZG-A9N3-01A-11R-A41O-07,TCGA-ZG-A9ND-01A-11R-A41O-07,TCGA-ZG-A9NI-01A-11R-A41O-07,TCGA-ZJ-A8QO-01A-11R-A37O-07,TCGA-ZJ-A8QQ-01A-11R-A37O-07,TCGA-ZJ-A8QR-01A-11R-A37O-07,TCGA-ZJ-AAX4-01A-11R-A42T-07,TCGA-ZJ-AAX8-01A-11R-A42T-07,TCGA-ZJ-AAXA-01A-11R-A42T-07,TCGA-ZJ-AAXB-01A-11R-A42T-07,TCGA-ZJ-AAXD-01A-21R-A42T-07,TCGA-ZJ-AAXF-01A-31R-A42T-07,TCGA-ZJ-AAXI-01A-11R-A42T-07,TCGA-ZJ-AAXJ-01A-11R-A42T-07,TCGA-ZJ-AAXN-01A-11R-A42T-07,TCGA-ZJ-AAXT-01A-11R-A42T-07,TCGA-ZJ-AAXU-01A-11R-A42T-07,TCGA-ZJ-AB0H-01A-11R-A42T-07,TCGA-ZJ-AB0I-01A-11R-A42T-07,TCGA-ZP-A9CV-01A-11R-A38B-07,TCGA-ZP-A9CY-01A-11R-A38B-07,TCGA-ZP-A9CZ-01A-11R-A38B-07,TCGA-ZP-A9D0-01A-11R-A37K-07,TCGA-ZP-A9D1-01A-11R-A38B-07,TCGA-ZP-A9D4-01A-11R-A37K-07,TCGA-ZQ-A9CR-01A-11R-A39E-31,TCGA-ZS-A9CD-01A-11R-A37K-07,TCGA-ZS-A9CE-01A-11R-A37K-07,TCGA-ZS-A9CF-01A-11R-A38B-07,TCGA-ZS-A9CG-01A-11R-A37K-07,TCGA-ZX-AA5X-01A-11R-A42T-07
0,ENSG00000000003.13,TSPAN6,574547.913106,352295.183420,2.951616e+05,3.295796e+05,2.892687e+05,7.494595e+05,4.645098e+05,298791.309231,8.453340e+05,4.511614e+05,360295.490644,1.047494e+06,704844.508623,231735.834578,2.043793e+05,5.776964e+05,7.611386e+05,315981.544960,205054.436679,186455.212337,709336.737655,356558.245525,197995.582766,3.501956e+05,285214.731463,226075.453877,288578.344313,285088.135143,348652.528936,567607.232282,313974.737051,477398.820402,247895.404954,142356.832479,163691.542345,7.674740e+05,7.297196e+05,4.230035e+05,...,235681.089587,241581.147491,287519.901019,1.467568e+06,487823.460526,173542.500496,247600.197177,540024.563558,269359.168477,4.542648e+05,542430.469457,157115.897041,187045.147961,2.569599e+05,195867.398022,561462.783555,285043.489451,4.266435e+05,3.258353e+05,334230.936855,293132.068927,295272.105679,2.776241e+05,266093.063609,85705.686518,1.666617e+05,185271.485241,251306.750465,1.023321e+06,729882.226737,506727.350573,1.242413e+06,641113.746173,437787.923442,1.476626e+05,1.180925e+06,929227.524309,469275.804035,2.441192e+06,259853.002700
1,ENSG00000000005.5,TNMD,2263.137229,4592.369778,6.490261e+02,1.835591e+03,4.006171e+04,4.849570e+03,4.752696e+02,1539.325929,1.146327e+03,1.200924e+03,1008.649591,2.533550e+04,261.028793,3782.819903,2.741378e+03,9.714262e+03,1.932940e+04,0.000000,0.000000,0.000000,217.936145,0.000000,0.000000,2.389036e+03,396.015948,504.597938,1019.898214,0.000000,1282.639158,207.315758,179.099773,201.547177,0.000000,0.000000,0.000000,0.000000e+00,5.488524e+03,0.000000e+00,...,1066.869372,3153.283475,794.776727,6.350900e+02,0.000000,923.481124,1059.928349,241.986174,541.751428,0.000000e+00,483.360321,42465.234787,229.470033,

In [ ]:
gene_name = df_FPKM_UQ['gene_name']

In [ ]:
# explore the file 
df_FPKM_UQ.iloc[0:6, 0:10]
cols = df_FPKM_UQ.columns[2:].values.tolist()

In [ ]:
# read the file 
submitters_id_to_project_id = pd.read_csv("/content/drive/My Drive/FNL_TC1/submitters_id_to_project_id.tsv", 
                                          low_memory=False, sep="\t")


In [ ]:
# explore the contents
submitters_id_to_project_id.columns
submitters_id_to_project_id


,submittedAliquot ID,mappedCaseId,mappedProject
0,TCGA-04-1331-01A-01R-1569-13,TCGA-04-1331,TCGA-OV
1,TCGA-04-1332-01A-01R-1564-13,TCGA-04-1332,TCGA-OV
2,TCGA-04-1338-01A-01R-1564-13,TCGA-04-1338,TCGA-OV
3,TCGA-04-1341-01A-01R-1564-13,TCGA-04-1341,TCGA-OV
4,TCGA-04-1343-01A-01R-1564-13,TCGA-04-1343,TCGA-OV
...,...,...,...
4495,TCGA-ZS-A9CD-01A-11R-A37K-07,TCGA-ZS-A9CD,TCGA-LIHC
4496,TCGA-ZS-A9CE-01A-11R-A37K-07,TCGA-ZS-A9CE,TCGA-LIHC
4497,TCGA-ZS-A9CF-01A-11R-A38B-07,TCGA-ZS-A9CF,TCGA-LIHC
4498,TCGA-ZS-A9CG-01A-11R-A37K-07,TCGA-ZS-A9CG,TCGA-LIHC


In [ ]:
# There are 15 cancer types. 
submitters_id_to_project_id.mappedProject.value_counts()

TCGA-PRAD    300
TCGA-KIRC    300
TCGA-HNSC    300
TCGA-OV      300
TCGA-LGG     300
TCGA-LUSC    300
TCGA-LIHC    300
TCGA-THCA    300
TCGA-BLCA    300
TCGA-STAD    300
TCGA-UCEC    300
TCGA-CESC    300
TCGA-COAD    300
TCGA-LUAD    300
TCGA-BRCA    300
Name: mappedProject, dtype: int64

In [ ]:
# Time elapsed to transpose: ~100 seconds on a modern laptop
dft_FPKM_UQ = df_FPKM_UQ.T

In [ ]:
# dft_FPKM_UQ.columns = dft_FPKM_UQ.loc['gene_name',:]

In [ ]:
print('Pre df', df_FPKM_UQ.shape)
print('After df Transpose', dft_FPKM_UQ.shape)

dft_FPKM_UQ[0:2]
dft_FPKM_UQ.index[0:2]

Pre df (60483, 4502)
After df Transpose (4502, 60483)


Index(['Unnamed: 0', 'gene_name'], dtype='object')

In [ ]:
# We are removing the first two rows (ESG*** and gene_name rows) 
# and saving them in dftm_FPM_UQ

print('Dimension of dft_FPKM_UQ :', dft_FPKM_UQ.shape)
dftm_FPKM_UQ = dft_FPKM_UQ.drop(dft_FPKM_UQ.index[0:2], axis=0)
print('Dimension of dftm_FPKM_UQ :', dftm_FPKM_UQ.shape)

Dimension of dft_FPKM_UQ : (4502, 60483)
Dimension of dftm_FPKM_UQ : (4500, 60483)


In [ ]:
dftm_FPKM_UQ

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,60443,60444,60445,60446,60447,60448,60449,60450,60451,60452,60453,60454,60455,60456,60457,60458,60459,60460,60461,60462,60463,60464,60465,60466,60467,60468,60469,60470,60471,60472,60473,60474,60475,60476,60477,60478,60479,60480,60481,60482
TCGA-04-1331-01A-01R-1569-13,574548,2263.14,983212,69718,54834.9,19718.1,175853,735123,38662.4,233190,30267.4,44557.6,210705,111871,826579,1925.14,210494,15383.1,123452,117745,2755.99,375245,495768,1.04871e+06,30118.3,44497.5,1117.34,1356.75,47876.5,695911,117716,257224,314105,12418.6,10752.9,175068,122596,117724,53470.3,195113,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
TCGA-04-1332-01A-01R-1564-13,352295,4592.37,663107,39745.4,36553.5,41147.1,241313,396423,37567,128693,30477.8,109417,103187,58024.2,486429,287707,105600,16938.6,62269.8,49997.3,1553.46,1.27161e+06,445869,2.49741e+06,77640.3,8056.26,2456.26,2823.71,136267,694851,60828.3,561595,263618,5911.73,15111.8,56940.6,350832,48062,58467.9,93396.4,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
TCGA-04-1338-01A-01R-1564-13,295162,649.026,1.21115e+06,57385.5,33097.4,58051.8,228615,346066,105567,408267,24432.1,115338,115917,123568,198117,63797.7,123545,5642.74,81688,105241,823.3,568350,345090,1.64758e+06,27127.8,2.40384e+06,961.299,2394.4,101747,586766,100262,542740,187850,598.558,5834.12,44865.2,185388,44075.5,69176.2,81863.6,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
TCGA-04-1341-01A-01R-1564-13,329580,1835.59,1.08437e+06,33812.3,24516.1,22330.6,42134.4,895558,56178,83847.3,22439.9,49770.9,291485,130180,1.11084e+06,38776,55126.9,5712.57,29593.5,73265.4,1338.87,1.21616e+06,582667,1.91073e+06,6559.49,1.21875e+06,792.974,2513.01,99429.7,1.17055e+06,151002,3.51457e+06,450005,2486.38,8250.09,62084.9,134241,39666.3,38604.1,54520.7,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
TCGA-04-1343-01A-01R-1564-13,289269,40061.7,2.44837e+06,26399.5,18248,49610,74761.1,571992,71951.9,98726.4,36465.7,109813,157905,82461.5,103327,5700.12,116968,17449.2,71768.8,48581,2131.12,3.17291e+06,818049,1.81917e+06,20468.2,185446,106.339,1887.19,129547,788571,99915.4,777468,245228,1489.78,9611.39,148251,138969,37635.8,19533.8,77930.7,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TCGA-ZS-A9CD-01A-11R-A37K-07,1.18093e+06,0,1.01139e+06,67877.2,15005.7,50527.3,6.21536e+06,1.47373e+06,459656,167488,13957.7,27150.1,426845,106741,245971,208.178,55827.4,65934.5,51442.4,24543.4,1257.28,758321,1.61776e+06,1.71501e+06,5536.71,1390.99,271.857,444.373,96990.5,1.08691e+06,181681,7.92674e+06,215939,14599.8,2651.62,23382.1,286811,72482.6,43521.2,118602,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
TCGA-ZS-A9CE-01A-11R-A37K-07,929228,0,869800,95607.5,17188.6,9352.12,7.61121e+06,196838,354465,138074,25074.6,43558,327242,229287,102038,224.39,82824.7,74744,86664.3,40138.3,100.385,697466,554237,1.37962e+06,2184.33,136105,0,319.32,313588,1.14904e+06,217825,9.58148e+06,168327,11221,1905.42,311033,2.47079e+06,103188,53156.9,111834,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
TCGA-ZS-A9CF-01A-11R-A38B-07,469276,476.683,516938,110051,34469.4,37334.7,5.95811e+06,427832,323833,154861,8386.06,47252.1,182218,62538.6,220251,810.982,52635,52746.4,53069,60261.2,483.744,664563,478522,1.59895e+06,2067.61,3211.13,4942.24,164.868,178119,566492,136210,32648.5,277358,22969.9,2907.62,35619.2,308522,91571.9,

In [ ]:
# view few entries.
dftm_FPKM_UQ.iloc[0:3, 0:15]
dft_FPKM_UQ.iloc[0:5,0:15]
dftm_FPKM_UQ.index

Index(['TCGA-04-1331-01A-01R-1569-13', 'TCGA-04-1332-01A-01R-1564-13',
       'TCGA-04-1338-01A-01R-1564-13', 'TCGA-04-1341-01A-01R-1564-13',
       'TCGA-04-1343-01A-01R-1564-13', 'TCGA-04-1347-01A-01R-1564-13',
       'TCGA-04-1356-01A-01R-1569-13', 'TCGA-04-1357-01A-01R-1565-13',
       'TCGA-04-1362-01A-01R-1565-13', 'TCGA-04-1364-01A-01R-1565-13',
       ...
       'TCGA-ZP-A9CZ-01A-11R-A38B-07', 'TCGA-ZP-A9D0-01A-11R-A37K-07',
       'TCGA-ZP-A9D1-01A-11R-A38B-07', 'TCGA-ZP-A9D4-01A-11R-A37K-07',
       'TCGA-ZQ-A9CR-01A-11R-A39E-31', 'TCGA-ZS-A9CD-01A-11R-A37K-07',
       'TCGA-ZS-A9CE-01A-11R-A37K-07', 'TCGA-ZS-A9CF-01A-11R-A38B-07',
       'TCGA-ZS-A9CG-01A-11R-A37K-07', 'TCGA-ZX-AA5X-01A-11R-A42T-07'],
      dtype='object', length=4500)

In [ ]:
# Extract the Submitter ID from the index and attach 
# it as a column called Submitter ID

dftm_FPKM_UQ['submitter_id'] = dftm_FPKM_UQ.index
dftm_FPKM_UQ
dftm_FPKM_UQ = dftm_FPKM_UQ.reset_index(drop=True)

dftm_FPKM_UQ.iloc[0:3, 0:12]
dft_FPKM_UQ.iloc[2:5,0:12]

dftm_FPKM_UQ['submitter_id']
dftm_FPKM_UQ.iloc[0:5,60483]

print(submitters_id_to_project_id.iloc[0:3,0:3])
sid_list = submitters_id_to_project_id['submittedAliquot ID'].values.tolist()
type(sid_list)
sid_list.index('TCGA-04-1338-01A-01R-1564-13')


            submittedAliquot ID  mappedCaseId mappedProject
0  TCGA-04-1331-01A-01R-1569-13  TCGA-04-1331       TCGA-OV
1  TCGA-04-1332-01A-01R-1564-13  TCGA-04-1332       TCGA-OV
2  TCGA-04-1338-01A-01R-1564-13  TCGA-04-1338       TCGA-OV


2

In [ ]:
# explore the file 
dftm_FPKM_UQ.shape

dftm_FPKM_UQ['Project_id'] = ' '
for idx, val in dftm_FPKM_UQ['submitter_id'].items():
    temp_sid = sid_list.index(val)
    dftm_FPKM_UQ['Project_id'][idx] = submitters_id_to_project_id['mappedProject'][temp_sid]


In [ ]:
dftm_FPKM_UQ[['Project_id','submitter_id']].iloc[[1,100,500,1000,2000,3000,4000],]


,Project_id,submitter_id
1,TCGA-OV,TCGA-04-1332-01A-01R-1564-13
100,TCGA-OV,TCGA-13-1411-01A-01R-1565-13
500,TCGA-BLCA,TCGA-4Z-AA7S-01A-11R-A39I-07
1000,TCGA-UCEC,TCGA-A5-A7WJ-01A-12R-A34R-07
2000,TCGA-BLCA,TCGA-CF-A1HS-01A-11R-A13Y-07
3000,TCGA-PRAD,TCGA-EJ-5530-01A-01R-1580-07
4000,TCGA-LGG,TCGA-P5-A5EU-01A-11R-A27Q-07


In [ ]:
# checking few random project_id and submitters_ids
dftm_FPKM_UQ[['Project_id','submitter_id']].iloc[[1,100,500,1000,2000,3000,4000],]

print(dftm_FPKM_UQ.shape)
dftm_FPKM_UQ.drop(['submitter_id'], axis = 1,  inplace=True)
print(dftm_FPKM_UQ.shape)
print(len(sid_list))

(4500, 60485)
(4500, 60484)
4500


In [ ]:
# ## Final check before moving on
dftm_FPKM_UQ.Project_id.value_counts()

print( 'before', dftm_FPKM_UQ['Project_id'][0:10] )
le = preprocessing.LabelEncoder()



before 0    TCGA-OV
1    TCGA-OV
2    TCGA-OV
3    TCGA-OV
4    TCGA-OV
5    TCGA-OV
6    TCGA-OV
7    TCGA-OV
8    TCGA-OV
9    TCGA-OV
Name: Project_id, dtype: object


In [ ]:
dftm_FPKM_UQ.to_csv('/content/drive/My Drive/FNL_TC1/dftm_FPKM_UQ.tsv', sep='\t', index=False, header = False)

In [ ]:
Project_id = dftm_FPKM_UQ['Project_id']
Project_id

0         TCGA-OV
1         TCGA-OV
2         TCGA-OV
3         TCGA-OV
4         TCGA-OV
          ...    
4495    TCGA-LIHC
4496    TCGA-LIHC
4497    TCGA-LIHC
4498    TCGA-LIHC
4499    TCGA-CESC
Name: Project_id, Length: 4500, dtype: object

In [ ]:
# Create a label (category) encoder object
dftm_FPKM_UQ['Project_id'] = le.fit_transform(dftm_FPKM_UQ.Project_id.values)

print( 'end', dftm_FPKM_UQ['Project_id'][0:10] )

print(len(le.classes_))
print(le.classes_)

dftm_FPKM_UQ['Project_id'].value_counts()
dftm_FPKM_UQ.shape
dftm_FPKM_UQ.columns

end 0    10
1    10
2    10
3    10
4    10
5    10
6    10
7    10
8    10
9    10
Name: Project_id, dtype: int64
15
['TCGA-BLCA' 'TCGA-BRCA' 'TCGA-CESC' 'TCGA-COAD' 'TCGA-HNSC' 'TCGA-KIRC'
 'TCGA-LGG' 'TCGA-LIHC' 'TCGA-LUAD' 'TCGA-LUSC' 'TCGA-OV' 'TCGA-PRAD'
 'TCGA-STAD' 'TCGA-THCA' 'TCGA-UCEC']


Index([           0,            1,            2,            3,            4,
                  5,            6,            7,            8,            9,
       ...
              60474,        60475,        60476,        60477,        60478,
              60479,        60480,        60481,        60482, 'Project_id'],
      dtype='object', length=60484)

In [ ]:
outcome = pd.DataFrame()
outcome['Project_id_name'] = Project_id
outcome['Project_id'] = dftm_FPKM_UQ['Project_id']
outcome

,Project_id_name,Project_id
0,TCGA-OV,10
1,TCGA-OV,10
2,TCGA-OV,10
3,TCGA-OV,10
4,TCGA-OV,10
...,...,...
4495,TCGA-LIHC,7
4496,TCGA-LIHC,7
4497,TCGA-LIHC,7
4498,TCGA-LIHC,7


In [ ]:
# outcome.to_csv('/content/drive/My Drive/FNL_TC1/TC1-outcome-data15-projectname.tsv', sep='\t', index=False, header = True)

In [ ]:
# Use to_categorical on your labels
features = dftm_FPKM_UQ.drop(['Project_id'], axis=1)
# pandas.core.series.Series
outcome = dftm_FPKM_UQ.Project_id
print(outcome)
#outcome.to_csv('/content/drive/My Drive/FNL_TC1/TC1-outcome-data15.tsv', sep='\t', index=False, header = False)

0       10
1       10
2       10
3       10
4       10
        ..
4495     7
4496     7
4497     7
4498     7
4499     2
Name: Project_id, Length: 4500, dtype: int64


In [ ]:
features.columns=gene_name

In [ ]:
features

gene_name,TSPAN6,TNMD,DPM1,SCYL3,C1orf112,FGR,CFH,FUCA2,GCLC,NFYA,STPG1,NIPAL3,LAS1L,ENPP4,SEMA3F,CFTR,ANKIB1,CYP51A1,KRIT1,RAD52,MYH16,BAD,LAP3,CD99,HS3ST1,AOC1,WNT16,HECW1,MAD1L1,LASP1,SNX11,TMEM176A,M6PR,KLHL13,CYP26B1,ICA1,DBNDD1,ALS2,CASP10,CFLAR,...,PLCXD1,WASH6P,WASIR1,IL3RA,SHOX,ASMT,AKAP17A,CSF2RA,CRLF2,ZBED1,RNA5SP498,TRPC6P,RP13-297E16.4,DHRSX-IT1,CD99P1,RPL14P5,LINC00685,DDX11L16,TCEB1P24,LL0YNC03-29C1.1,KRT18P53,LINC00102,RP13-297E16.5,FABP5P13,ASMTL-AS1,LINC00106,DPH3P2,RP11-309M23.1,AMDP1,BX649553.1,BX649553.2,BX649553.3,BX649553.4,MIR3690,AJ271736.10,Metazoa_SRP,AJ271736.1,MIR6089,RP13-465B17.5,RP13-465B17.4
0,574548,2263.14,983212,69718,54834.9,19718.1,175853,735123,38662.4,233190,30267.4,44557.6,210705,111871,826579,1925.14,210494,15383.1,123452,117745,2755.99,375245,495768,1.04871e+06,30118.3,44497.5,1117.34,1356.75,47876.5,695911,117716,257224,314105,12418.6,10752.9,175068,122596,117724,53470.3,195113,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,352295,4592.37,663107,39745.4,36553.5,41147.1,241313,396423,37567,128693,30477.8,109417,103187,58024.2,486429,287707,105600,16938.6,62269.8,49997.3,1553.46,1.27161e+06,445869,2.49741e+06,77640.3,8056.26,2456.26,2823.71,136267,694851,60828.3,561595,263618,5911.73,15111.8,56940.6,350832,48062,58467.9,93396.4,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,295162,649.026,1.21115e+06,57385.5,33097.4,58051.8,228615,346066,105567,408267,24432.1,115338,115917,123568,198117,63797.7,123545,5642.74,81688,105241,823.3,568350,345090,1.64758e+06,27127.8,2.40384e+06,961.299,2394.4,101747,586766,100262,542740,187850,598.558,5834.12,44865.2,185388,44075.5,69176.2,81863.6,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,329580,1835.59,1.08437e+06,33812.3,24516.1,22330.6,42134.4,895558,56178,83847.3,22439.9,49770.9,291485,130180,1.11084e+06,38776,55126.9,5712.57,29593.5,73265.4,1338.87,1.21616e+06,582667,1.91073e+06,6559.49,1.21875e+06,792.974,2513.01,99429.7,1.17055e+06,151002,3.51457e+06,450005,2486.38,8250.09,62084.9,134241,39666.3,38604.1,54520.7,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,289269,40061.7,2.44837e+06,26399.5,18248,49610,74761.1,571992,71951.9,98726.4,36465.7,109813,157905,82461.5,103327,5700.12,116968,17449.2,71768.8,48581,2131.12,3.17291e+06,818049,1.81917e+06,20468.2,185446,106.339,1887.19,129547,788571,99915.4,777468,245228,1489.78,9611.39,148251,138969,37635.8,19533.8,77930.7,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4495,1.18093e+06,0,1.01139e+06,67877.2,15005.7,50527.3,6.21536e+06,1.47373e+06,459656,167488,13957.7,27150.1,426845,106741,245971,208.178,55827.4,65934.5,51442.4,24543.4,1257.28,758321,1.61776e+06,1.71501e+06,5536.71,1390.99,271.857,444.373,96990.5,1.08691e+06,181681,7.92674e+06,215939,14599.8,2651.62,23382.1,286811,72482.6,43521.2,118602,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4496,929228,0,869800,95607.5,17188.6,9352.12,7.61121e+06,196838,354465,138074,25074.6,43558,327242,229287,102038,224.39,82824.7,74744,86664.3,40138.3,100.385,697466,554237,1.37962e+06,2184.33,136105,0,319.32,313588,1.14904e+06,217825,9.58148e+06,168327,11221,1905.42,311033,2.47079e+06,103188,53156.9,111834,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4497,469276,476.683,516938,110051,34469.4,37334.7,5.95811e+06,427832,323833,154861,8386.06,47252.1,182218,62538.6,220251,810.982,52635,52746.4,53069,60261.2,483.744,664563,478522,1.59895e+06,2067.61,3211.13,4942.24,164.868,17811

In [ ]:
# features.to_csv('/content/drive/My Drive/FNL_TC1/TC1-features-data15.tsv', sep='\t', index=False, header = True)

# Scaling 

In [ ]:
features = pd.read_csv("/content/drive/My Drive/FNL_TC1/TC1-features-data15.tsv", low_memory=False, sep="\t")

In [ ]:
sfeatures = features.div(features.sum(axis=1), axis=0)
sfeatures = sfeatures * 1000000

# log scaling
sfeatures1 = sfeatures.astype(np.float64).apply(np.log10)

# since we have negative numbers
sfeatures1[sfeatures1 < 0] = 0

In [ ]:
# just making sure
sfeatures1.isnull().sum().sum()

# just checking
print('features few entries')
features.iloc[0:5,0:5]

print('sfeatures few entries')
sfeatures.iloc[0:5,0:5]

print('sfeatures1 few entries')
sfeatures1.iloc[0:5,0:5]

features few entries
sfeatures few entries
sfeatures1 few entries


,TSPAN6,TNMD,DPM1,SCYL3,C1orf112
0,1.829525,0.000000,2.062846,0.913543,0.809255
1,1.541309,0.000000,1.815986,0.593689,0.557331
2,1.282789,0.000000,1.895928,0.571531,0.332523
3,1.388260,0.000000,1.905475,0.399374,0.259750
4,1.389160,0.530588,2.316735,0.349455,0.189075


In [ ]:
sfeatures1.to_csv('/content/drive/My Drive/FNL_TC1/TC1-S1-data15-genename.tsv', sep='\t', index=False, header = True)

In [ ]:
# Read features and output files 
# TC1data15 = pd.read_csv("TC1-S1-data15.tsv", sep="\t", low_memory = False)
TC1data15 = sfeatures1

In [ ]:
outcome = pd.read_csv('TC1-outcome-data15.tsv', sep='\t',names='y')

In [ ]:
#outcome = outcome.values
outcome = outcome['y']

In [ ]:
# sfeatures1.to_csv('TC1-S1-data15.tsv', sep='\t', index=False)

In [ ]:
def encode(data): 
    print('Shape of data (BEFORE encode): %s' % str(data.shape))
    encoded = to_categorical(data)
    print('Shape of data (AFTER  encode): %s\n' % str(encoded.shape))
    return encoded



In [ ]:
# from keras.utils import to_categorical
# outcome = encode(outcome)